In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [3]:
# Load data
train_data = datasets.FashionMNIST('.', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST('.', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

100%|██████████| 26.4M/26.4M [00:01<00:00, 18.7MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 277kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 5.16MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 18.2MB/s]


In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.net(x)

In [5]:
# Initialize model, optimizer, and loss function
model = Net().to(device)  # Move model to device
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [6]:
# Training loop
num_epochs = 10  # Define number of epochs
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)  # Move data to device
        pred = model(x)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

    # Evaluation on test data
    model.eval() # Set model to evaluation mode
    with torch.no_grad():
        correct = 0
        total = 0
        for x, y in test_loader:
            x, y = x.to(device), y.to(device) # Move data to device
            outputs = model(x)
            _, predicted = torch.max(outputs.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()

        print(f'Accuracy on test set: {100 * correct / total}%')

Epoch 1, Loss: 0.5472149133269213
Accuracy on test set: 83.26%
Epoch 2, Loss: 0.40476322914364493
Accuracy on test set: 85.3%
Epoch 3, Loss: 0.36075703666281345
Accuracy on test set: 86.48%
Epoch 4, Loss: 0.33426614359084733
Accuracy on test set: 86.54%
Epoch 5, Loss: 0.3144206692462664
Accuracy on test set: 87.26%
Epoch 6, Loss: 0.29954594391177713
Accuracy on test set: 86.58%
Epoch 7, Loss: 0.2873162974093133
Accuracy on test set: 86.91%
Epoch 8, Loss: 0.2738991934559874
Accuracy on test set: 88.13%
Epoch 9, Loss: 0.26546317580412193
Accuracy on test set: 87.85%
Epoch 10, Loss: 0.2535619937709527
Accuracy on test set: 87.96%
